In [36]:
import zipfile
import os
import xarray as xr
import pandas as pd
import numpy as np
import hashlib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score,mean_absolute_percentage_error, accuracy_score
from IPython.display import display, HTML


In [2]:
!pip install h5netcdf
!pip install netcdf4
!pip install scipy

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [140]:
from IPython.display import display, HTML

display(HTML("<b>To predict GHI (Global Horizontal Irradiance) at 15-minute intervals, we need to extract features from both the satellite images and time-based information. Here are potential features you can extract: 1. Satellite Data-Based Features. Since we are predicting GHI, which is related to solar radiation, solar angles (like Sun Azimuth and Sun Elevation) are critical, as they directly affect irradiance. Similarly, satellite bands (e.g., MIR, SWIR, TIR1) capture various parts of the spectrum relevant to solar radiation.</b>"))


In [141]:
display(HTML("<b>below function return features corresponding one folder</b>"))

In [3]:
def extract_features_from_file(file_path):
    # Open the dataset
    ds = xr.open_dataset(file_path)

    # Get time
    time = ds['time'].values  

    # Features
    sun_azimuth = ds['Sun_Azimuth'].mean(dim=["latitude", "longitude"]).values
    sun_elevation = ds['Sun_Elevation'].mean(dim=["latitude", "longitude"]).values
    img_mir_mean = ds['IMG_MIR'].mean(dim=["latitude", "longitude"]).values
    img_mir_std = ds['IMG_MIR'].std(dim=["latitude", "longitude"]).values
    grey_count_mean = ds['IMG_MIR_RADIANCE'].mean(dim="GreyCount").values
    grey_count_std = ds['IMG_MIR_RADIANCE'].std(dim="GreyCount").values

    # Time features
    hour_of_day = np.array([t.hour for t in pd.to_datetime(time)])
    day_of_year = np.array([t.timetuple().tm_yday for t in pd.to_datetime(time)])
    hour_sin = np.sin(2 * np.pi * hour_of_day / 24)
    hour_cos = np.cos(2 * np.pi * hour_of_day / 24)
    day_sin = np.sin(2 * np.pi * day_of_year / 365)
    day_cos = np.cos(2 * np.pi * day_of_year / 365)

    # Collect feature rows for each time step
    rows = []
    for i in range(len(time)):
        rows.append({
            "filename": os.path.basename(file_path),
            "time": time[i],
            "sun_azimuth": sun_azimuth[i],
            "sun_elevation": sun_elevation[i],
            "img_mir_mean": img_mir_mean[i],
            "img_mir_std": img_mir_std[i],
            "grey_count_mean": grey_count_mean[i],
            "grey_count_std": grey_count_std[i],
            "hour_sin": hour_sin[i],
            "hour_cos": hour_cos[i],
            "day_sin": day_sin[i],
            "day_cos": day_cos[i]
        })
    
    return rows

def process_folders_and_extract_features(all_folders):
    rows = []  # List to hold all rows
    for folder in all_folders:
        for filename in os.listdir(folder):
            if filename.endswith(".nc"):
                file_path = os.path.join(folder, filename)
                # Extract features for the current file
                rows.extend(extract_features_from_file(file_path))  # Append rows
    return rows



In [143]:
display(HTML("<b>using below code i extracted .nc folder files. don't run below file</b>"))

In [144]:
with zipfile.ZipFile("Jun_Output.zip", 'r') as zip_ref:
    zip_ref.extractall("Jun-data")  # Extract to this directory

with zipfile.ZipFile("Jul_Output.zip", 'r') as zip_ref:
    zip_ref.extractall("Jul-data")  # Extract to this directory 

with zipfile.ZipFile("Aug_Output.zip", 'r') as zip_ref:
    zip_ref.extractall("Aug-data")  # Extract to this directory



In [145]:
display(HTML('<b>here storing 3 month .nc file information in all_folders variable</b>')) 

In [4]:
folder_Y_Jun = os.path.join("Jun-data", "Insat_Jun_Output_Kopal_3D")
folder_Y_Jul = os.path.join("Jul-data", "Insat_Jul_Output_Kopal_3DR_S")
folder_Y_Aug = os.path.join("Aug-data", "Insat_Aug_Output_Kopal_3DR_S")

# Combine all folders 
all_folders = [folder_Y_Jun, folder_Y_Jul, folder_Y_Aug] 

# Normalize file names to detect duplicates (ignoring case)
unique_files = set()
duplicates = []

In [147]:
display(HTML('<b>cleaning data- here removing duplicates from all folders.</b>'))

In [5]:

# Check for duplicates in each folder
for folder in all_folders:
    for file in os.listdir(folder):  # Loop through files in each folder
        if file.endswith('.nc'):
            lower_name = file.lower()
            if lower_name in unique_files:
                duplicates.append(os.path.join(folder, file))  # Store full path of duplicate
            else:
                unique_files.add(lower_name)

# Remove duplicate files
for dup_file in duplicates:
    print(f"Trying to remove duplicate file: {dup_file}") 
    try: 
        os.remove(dup_file)
        print(f"Removed: {dup_file}")  
    except FileNotFoundError:
        print(f"File not found, skipping: {dup_file}") 
    except PermissionError:
        print(f"Permission denied, skipping: {dup_file}")


In [149]:
display(HTML('<b>printing folder names</b>'))

In [6]:
for folder in all_folders:
    print(folder,"\n") 

Jun-data\Insat_Jun_Output_Kopal_3D 

Jul-data\Insat_Jul_Output_Kopal_3DR_S 

Aug-data\Insat_Aug_Output_Kopal_3DR_S 



In [157]:
display(HTML('<b>extracting revelant features that are necessary for predcting GHI values.</b>'))

In [54]:
feature_rows = process_folders_and_extract_features(all_folders) 

# Convert the list of rows into a DataFrame
features_df = pd.DataFrame(feature_rows)

# Display the DataFrame
print(features_df) 
#scaler = MinMaxScaler()

# Normalize the data
#x = scaler.fit_transform(features_df)

                          filename                time  sun_azimuth  \
0     INSAT_3DR_Kopal_10Jun2024.nc 2024-06-10 04:15:00    75.787994   
1     INSAT_3DR_Kopal_10Jun2024.nc 2024-06-10 04:45:00    74.839592   
2     INSAT_3DR_Kopal_10Jun2024.nc 2024-06-10 05:15:00    72.220001   
3     INSAT_3DR_Kopal_10Jun2024.nc 2024-06-10 05:45:00    65.788406   
4     INSAT_3DR_Kopal_10Jun2024.nc 2024-06-10 06:15:00    46.572399   
...                            ...                 ...          ...   
4832   INSAT_3DS_Kopal_9AUG2024.nc 2024-08-09 16:30:00   314.170807   
4833   INSAT_3DS_Kopal_9AUG2024.nc 2024-08-09 17:00:00   321.773193   
4834   INSAT_3DS_Kopal_9AUG2024.nc 2024-08-09 17:30:00   331.105194   
4835   INSAT_3DS_Kopal_9AUG2024.nc 2024-08-09 18:00:00   342.200012   
4836   INSAT_3DS_Kopal_9AUG2024.nc 2024-08-09 18:30:00   354.787201   

      sun_elevation  img_mir_mean  img_mir_std  grey_count_mean  \
0         56.889206    880.719971    14.334629         0.161563   
1         63.

In [120]:
display(HTML('<b>Extracting labels from excel sheet (only Jun-Aug labels) - GHI values for corresponding features</b>'))

In [8]:
GHI_df = pd.read_excel("Sample Dataset - ML Assignment.xlsx")
GHI_train = GHI_df.iloc[954:9691]
GHI_train = GHI_train.drop(columns='Unnamed: 1')

# Clean column names by stripping any extra spaces 
GHI_train.columns = GHI_train.columns.str.strip()

# Now rename the 'Date' column to 'time'
GHI_train.rename(columns={'Date': 'time','Observed GHI' : 'GHI'}, inplace=True)

print(GHI_train.head()) 

                   time  GHI
954 2024-06-01 00:30:00 -2.0
955 2024-06-01 00:45:00 -2.0
956 2024-06-01 01:00:00 -2.0
957 2024-06-01 01:15:00 -2.0
958 2024-06-01 01:30:01 -2.0


In [122]:
display(HTML('<b>merging features and GHI labels by matching time. </b>'))

In [13]:
features_df['time'] = pd.to_datetime(features_df['time'])
GHI_train['time'] = pd.to_datetime(GHI_train['time'])

# Merge the GHI data with the feature DataFrame
merged_df = pd.merge(features_df, GHI_train, on='time')  # Ensure 'time' is the common column
x = merged_df.drop(columns=['GHI', 'time','filename'])  # only features
y = merged_df['GHI'] #only  labels- GHI


In [159]:
display(HTML('<b>Normalizing features(x) using minmaxscaler</b>'))

In [14]:
scaler = MinMaxScaler()

# Normalize the data
x = scaler.fit_transform(x)
print(x)

[[0.2043414  0.62444031 0.76122219 ... 1.         0.89641932 0.12077157]
 [0.20172962 0.70045499 0.72727986 ... 1.         0.89641932 0.12077157]
 [0.17680385 0.84951648 0.55213424 ... 0.8392127  0.89641932 0.12077157]
 ...
 [0.78025784 0.09375485 0.7655536  ... 0.         0.17890078 0.43606758]
 [0.93800494 0.         0.77634279 ... 0.66666667 0.17890078 0.43606758]
 [0.9726684  0.         0.78429675 ... 0.66666667 0.17890078 0.43606758]]


In [15]:
print(features_df['time'].min(), features_df['time'].max())
print(GHI_train['time'].min(), GHI_train['time'].max()) 


2024-06-01 04:00:00 2024-08-31 17:15:00
2024-06-01 00:30:00 2024-09-01 00:56:10


In [161]:
display(HTML("<b>Spillting  x and y into train and test data</b>"))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
print(y.shape)
print(x.shape)


(3238,)
(3238, 10)


In [31]:
display(HTML("<b>Below we are taking sep data</b>")) 

In [22]:
with zipfile.ZipFile("Sep_Validation_Data.zip", 'r') as zip_ref:
    zip_ref.extractall("Sep-data")  # Extract to this directory


In [32]:
display(HTML("<b>Extracting features of sep data</b>"))

In [23]:
folder_Y_Sep= os.path.join("Sep-data", "INSAT_Validation_Data")
p=[] 
for filename in os.listdir(folder_Y_Sep):
            if filename.endswith(".nc"):
                file_path = os.path.join(folder_Y_Sep, filename)
                # Extract features for the current file
                p.extend(extract_features_from_file(file_path))  
            
df = pd.DataFrame(p) 
print(df) 

                  filename                time  sun_azimuth  sun_elevation  \
0    INSAT_3DR_1SEP2024.nc 2024-09-01 04:45:00   106.274796      61.540794   
1    INSAT_3DR_1SEP2024.nc 2024-09-01 05:15:00   113.290390      68.331596   
2    INSAT_3DR_1SEP2024.nc 2024-09-01 05:45:00   125.239601      74.650002   
3    INSAT_3DR_1SEP2024.nc 2024-09-01 06:15:00   148.539612      79.702789   
4    INSAT_3DR_1SEP2024.nc 2024-09-01 06:45:00   189.170395      81.304001   
..                     ...                 ...          ...            ...   
352  INSAT_3DS_7SEP2024.nc 2024-09-07 15:00:00   289.625183       0.500000   
353  INSAT_3DS_7SEP2024.nc 2024-09-07 15:30:00   293.658386       0.500000   
354  INSAT_3DS_7SEP2024.nc 2024-09-07 16:00:00   298.650787       0.500000   
355  INSAT_3DS_7SEP2024.nc 2024-09-07 16:30:00   304.994812       0.500000   
356  INSAT_3DS_7SEP2024.nc 2024-09-07 17:00:00   313.317200       0.500000   

     img_mir_mean  img_mir_std  grey_count_mean  grey_count_std

C:\Users\abc\AppData\Roaming\Python\Python312\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


In [40]:
display(HTML("<b>Extracting labels of sep month from given Excel sheet</b>"))

In [24]:
#GHI_df = pd.read_excel("Sample Dataset - ML Assignment.xlsx")
GHI_test = GHI_df.iloc[9691:]
GHI_test= GHI_test.drop(columns='Unnamed: 1')
print(GHI_test) 
# Clean column names by stripping any extra spaces 
GHI_test.columns = GHI_test.columns.str.strip() 

# Now rename the 'Date' column to 'time'
GHI_test.rename(columns={'Date': 'time','Observed GHI' : 'GHI'}, inplace=True) 
# Drop 'filename' column from features_df before merging
sep_df = df.drop(columns=['filename'])

# Ensure 'time' column is datetime format in both DataFrames
sep_df['time'] = pd.to_datetime(sep_df['time'])
GHI_test['time'] = pd.to_datetime(GHI_test['time'])

# Merge the GHI data with the feature DataFrame
merged_df = pd.merge(sep_df, GHI_test, on='time')  # Ensure 'time' is the common column
X = merged_df.drop(columns=['GHI', 'time'])  
sep_X= scaler.fit_transform(X) 
sep_Y= merged_df['GHI']  

                    Date   Observed GHI 
9691  2024-09-01 01:10:32            0.0
9692  2024-09-01 02:32:06            0.0
9693  2024-09-01 05:56:31            0.0
9694  2024-09-01 06:09:46            0.0
9695  2024-09-01 07:20:17            0.0
...                   ...            ...
10510 2024-09-09 23:00:00            0.0
10511 2024-09-09 23:15:01            0.0
10512 2024-09-09 23:30:00            0.0
10513 2024-09-09 23:45:00            0.0
10514 2024-09-10 00:00:00            0.0

[824 rows x 2 columns]


In [34]:
display(HTML("<b>Training 3 months features and its labels using different ML models</b>"))

In [39]:
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Support Vector Machine': SVR(),
    'Decision Tree': DecisionTreeRegressor(),
    'K-Nearest Neighbors': KNeighborsRegressor()
}

# Initialize a dictionary to store model performance
performance = {}

# Train and evaluate each model
for model_name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_pred_sep = model.predict(sep_X)
    
    # Evaluate performance (using Mean Absolute Error here)
    mae = mean_absolute_error(sep_Y, y_pred_sep)
    rmse = np.sqrt(mean_squared_error(sep_Y, y_pred_sep))
    r2 = r2_score(sep_Y, y_pred_sep) 
    mape=mean_absolute_percentage_error(sep_Y, y_pred_sep)
    
    # Store the performance result
    performance[model_name] = {'MAE': mae, 'RMSE': rmse, 'R^2': r2,'MAPE':mape} 
    # Store the performance result
    #performance[model_name] = mae 

print("Model Performance:\n")
for model_name, metrics in performance.items():
    print(f"{model_name}: MAE = {metrics['MAE']}, RMSE = {metrics['RMSE']}, R^2 = {metrics['R^2']}, MAPE={metrics['MAPE']}")
    print("\n") 

Model Performance:

Linear Regression: MAE = 114.89047978548452, RMSE = 151.7035660491518, R^2 = 0.6412663289966773, MAPE=2.4587363887551052e+16


Random Forest: MAE = 146.1275481167383, RMSE = 203.96923257220752, R^2 = 0.35150014088195414, MAPE=1.1846992718843752e+16


Support Vector Machine: MAE = 198.03107585598778, RMSE = 260.34097344109, R^2 = -0.056490536189091944, MAPE=3.223241448289863e+16


Decision Tree: MAE = 184.94769668737058, RMSE = 250.5003546953168, R^2 = 0.021868483473401867, MAPE=3970020975866821.5


K-Nearest Neighbors: MAE = 133.46883822699039, RMSE = 182.19872769865682, R^2 = 0.48254660317490483, MAPE=1041375826879657.0




In [50]:
display(HTML("<b>linear regression performance is better out off other ML algorithms. </b>"))
display(HTML("<b>Below are Linear Regression results </b>"))

In [49]:
Regression_model=LinearRegression()
Regression_model.fit(X_train, y_train)
performance_model = {} 
    
# Make predictions
final_predictions = Regression_model.predict(sep_X)

# Compute metrics
mae = mean_absolute_error(sep_Y, final_predictions)
rmse = np.sqrt(mean_squared_error(sep_Y, final_predictions))
r2 = r2_score(sep_Y, final_predictions)
mape = mean_absolute_percentage_error(sep_Y, final_predictions)

# Print or log results
print("Forecast Accuracy Metrics:")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"R²: {r2}")
print(f"MAPE: {mape}") 

Forecast Accuracy Metrics:
MAE: 114.89047978548452
RMSE: 151.7035660491518
R²: 0.6412663289966773
MAPE: 2.4587363887551052e+16


In [51]:
display(HTML("<b>printing actual and predicted labels of sep month</b>"))

In [52]:
df = pd.DataFrame({
    'Actual': sep_Y,
    'Predicted': final_predictions
}) 
print(df)

         Actual   Predicted
0    312.933333  557.075143
1    382.866667  495.073945
2    297.600000  496.386195
3    194.333333  405.639674
4    184.066667  407.413443
..          ...         ...
179  479.466667  427.755929
180  410.000000  515.016470
181  694.600000  589.694493
182  607.200000  521.448343
183  751.000000  410.831753

[184 rows x 2 columns]


In [53]:
display(HTML("<b>saving resutls in CSV fromat</b>"))

In [48]:
ds = xr.Dataset(
    {
        "forecast": (["time"], final_predictions),
        "actual": (["time"], sep_Y),
    },
    coords={"time": sep_Y.index} 
)
ds.to_netcdf("forecast_results.nc") 
print(ds) 

<xarray.Dataset>
Dimensions:   (time: 184)
Coordinates:
  * time      (time) int64 0 1 2 3 4 5 6 7 8 ... 176 177 178 179 180 181 182 183
Data variables:
    forecast  (time) float64 557.1 495.1 496.4 405.6 ... 515.0 589.7 521.4 410.8
    actual    (time) float64 312.9 382.9 297.6 194.3 ... 410.0 694.6 607.2 751.0
